# Footprint Simulator

In [13]:
import tkinter

from pathlib import Path
from tkinter import ttk

import floor_plan

class FootprintSimulator:
    N, W, E, S = tkinter.N, tkinter.W, tkinter.E, tkinter.S

    def __init__(self, root, path):
        """
        Parameters
        ----------
        root : tkinter.Tk
        path : pathlib.Path
            A data folder path that includes layout folders.
        
        Returns
        -------
        None
        """
        
        self.root = root
        self.path = path
        self.layout_path = Path()
        self.data_path = Path()
        
        self.root.title('Footprint Simulator')
        
        # widgets
        self.mainframe = ttk.Frame(root, padding = (3,3,12,12))

        self._time_l = ttk.Label(self.mainframe, text = "Time")  # l means label
        self._time = ttk.Label(self.mainframe, text = "Time")
        
        self._activity_l = ttk.Label(self.mainframe, text = "Activity")
        self._activity = ttk.Label(self.mainframe, text = "Activity")
        
        # self._figure = ttk.Frame(self.mainframe, borderwidth = 5, relief = "ridge", width = 200, height = 100)
        self._figure = tkinter.Canvas(self.mainframe, borderwidth = 5, relief = 'ridge', bg = 'white')
        
        self.layouts = [p for p in self.path.iterdir()]
        self.layouts.remove(path / '.ipynb_checkpoints')
        self.layouts_name = [p.name for p in self.layouts]
        self._layouts_name = tkinter.StringVar(value = self.layouts_name)
        self._layout_l = ttk.Label(self.mainframe, text = "Layout")
        self._layout = tkinter.Listbox(self.mainframe, listvariable = self._layouts_name, height = 3)
        self._layout.bind('<<ListboxSelect>>', self.show_data_list)
        
        self.data = []
        self.data_name = []
        self._data_name = tkinter.StringVar(value = self.data_name)
        self._data_l = ttk.Label(self.mainframe, text = "Data")
        self._data = tkinter.Listbox(self.mainframe, listvariable = self._data_name, height = 3)
        self._data.bind('<<ListboxSelect>>', self.draw_figure)
        
        self._simulation_l = ttk.Label(self.mainframe, text = "Simulation")
        self._simulation = ttk.Button(self.mainframe, text="Start")

        # grid geometry manager
        self.mainframe.grid(row = 0, column = 0, sticky = (self.N, self.S, self.E, self.W)) 
        self._time_l.grid(row = 0, column = 0)
        self._time.grid(row = 0, column = 1)
        self._activity_l.grid(row = 0, column = 2)
        self._activity.grid(row = 0, column = 3) 
        self._figure.grid(row = 1, column = 0, columnspan = 6, sticky = (self.N, self.S, self.E, self.W)) 
        self._layout_l.grid(row = 2, column = 0)
        self._layout.grid(row = 2, column = 1)
        self._data_l.grid(row = 2, column = 2)
        self._data.grid(row = 2, column = 3)
        self._simulation_l.grid(row = 2, column = 4)
        self._simulation.grid(row = 2, column = 5) 
        
        # resizable
        self.root.columnconfigure(0, weight = 1)
        self.root.rowconfigure(0, weight = 1)
        self.mainframe.columnconfigure(0, weight = 1)
        self.mainframe.columnconfigure(1, weight = 1)
        self.mainframe.columnconfigure(2, weight = 1)
        self.mainframe.columnconfigure(3, weight = 1)
        self.mainframe.rowconfigure(1, weight = 1)
        
    def show_data_list(self, *args):
        indexes = self._layout.curselection()
        if len(indexes) == 0:
            return
        index = int(indexes[0])
        self.layout_path = self.layouts[index]
        self.data = []
        for p in self.layout_path.iterdir():
            if p.is_dir() and p.name != '.ipynb_checkpoints':
                self.data.append(p)
        self.data_name = [p.name for p in self.data]
        self._data_name.set(self.data_name)
        self._figure.delete("all")
        
    @staticmethod
    def point2canvas(xy, reference, scale):
        """
        This converts a coordinate in the layout into the coordinate on the canvas.
        
        Parameters
        ----------
        xy : tuple of float
            xy = (x, y) is the target point in the layout.
        reference : tuple of float
            This is the reference point as the top left coordinate in the layout.
        scale : float
            length in the layout : length on the canvas = 1 : scale.
            
        
        Returns
        -------
        ret : tuple of float
            ret = (xx, yy) is the calculated coordinate on the canvas.
        """
        return (scale*(xy[0] - reference[0]), scale*(reference[1] - xy[1]))
        
        
        
        
        
        
    def draw_figure(self, *args):
        """
        This draws a layout and sensor arrangements on the canvas.
        """
        scale = 1  # length in the layout : length on the canvas = 1 : scale.
        indexes = self._data.curselection()
        index = int(indexes[0])
        self.data_path = self.data[index]
        
        fp = floor_plan.FloorPlan()
        fp.load_layout(self.layout_path)
        
        # reference is the reference point as the top left coordinate in the layout
        reference = (fp.Boundary[0][0] - fp.edge, fp.Boundary[1][1] + fp.edge)
        
        for room in fp.Toil_Bath:
            [x, y, L, W, name] = room
            xx, yy = self.point2canvas((x, y), reference, scale)
            LL, WW = scale * L , scale * W
            self._figure.create_rectangle(xx, yy, xx + LL, yy - WW, width = 3)
            self._figure.create_text(xx + LL / 2, yy - WW / 2, text = fp.Code[name])
            
        for room in fp.Furnitures:
            for furniture in room:
                [x, y, L, W, name] = furniture
                xx, yy = self.point2canvas((x, y), reference, scale)
                LL, WW = scale * L , scale * W
                self._figure.create_rectangle(xx, yy, xx + LL, yy - WW, width = 1)
                self._figure.create_text(xx + LL / 2, yy - WW / 2, text = fp.Code[name])
        for wall in fp.Walls:
            s = self.point2canvas((wall[0][0], wall[0][1]), reference, scale)
            e = self.point2canvas((wall[1][0], wall[1][1]), reference, scale)
            self._figure.create_line(s[0], s[1], e[0], e[1])
        for door in fp.Doors:
            s = self.point2canvas((door[0], door[1]), reference, scale)
            LL, WW = scale * door[2], scale * door[3]
            self._figure.create_line(s[0], s[1], s[0] + LL, s[1] - WW)


root = tkinter.Tk()
path = Path('./layout_data')
FootprintSimulator(root, path)
root.mainloop()

(-240, 260)
-190 0
50 260
-190 -165
50 425
40 -200
280 460
100 -160
340 420
950 30
1190 230
720 -90
960 350
830 -90
1070 350
830 150
1070 110
720 150
960 110
250 -90
490 350
280 170
520 90
515 90
755 170
605 130
845 130
535 40
775 220


In [ ]:
fp = floor_plan.FloorPlan()
fp.load_layout(Path('./layout_data/test_layout'))
print(fp.Toil_Bath)

X_lim = [fp.Boundary[0][0] - fp.edge, fp.Boundary[0][1] + fp.edge]
Y_lim = [fp.Boundary[1][0] - fp.edge, fp.Boundary[1][1] + fp.edge]
print(X_lim)
print(Y_lim)

In [ ]:
def draw_figure(self, *args):
    indexes = self._data.curselection()
    index = int(indexes[0])
    data_path = self.data[index]

    fp = floor_plan.FloorPlan()
    fp.load_layout(layout_path)

    house_js = Load.load_se_map(path_l[0], path_l[1], path_l[2])
    rooms, T_Bs, furnitures, doors, walls, self.lims = Load.dic2house(house_js)
    Fur_set, self.scale, self.IsROT, Fur_indexs = SP.layoutplot(Canvas, [640, 370], rooms, T_Bs, furnitures, doors,
                                                                walls, self.lims, showbg=False)
        
        
def realxy2can(real_size, x0, y0, scale, PW, PH, Rote, Is_Rect=True):
    # real_size is the [x, y, L, W], x0, y0 is the coordinate of center of the house
    # PW, PH is the size of canvas, Rote is Rotation_Flag
    # if Is_Rect is True, transfer for a rectangle; else, for a point
    if not Is_Rect:
        [x, y] = real_size
        canx, cany = int(PW/2+(x-x0)/scale), int(PH/2-(y-y0)/scale)
        return [canx, cany] if not Rote else [int(PW/2+(PH/2-cany)), int(PH/2-(PW/2-canx))]
    else:
        [x, y, L, W] = real_size
        xm, ym = x+L//2, y+W//2
        [canxm, canym] = realxy2can([xm, ym], x0, y0, scale, PW, PH, Rote, Is_Rect=False)
        [HcanL,HcanW] = [int(L/scale/2), int(W/scale/2)] if not Rote else [int(W/scale/2), int(L/scale/2)]
        return [canxm-HcanL, canym-HcanW, canxm+HcanL, canym+HcanW]


def canxy2real(can_size, x0, y0, scale, PW, PH, Rote):
    [canx, cany] = can_size
    if Rote: canx, cany = int((PW-PH)//2+cany), int((PH+PW)//2-canx)
    return [int((x0+(canx-PW//2)*scale)//g_L*g_L), int((y0+(PH//2-cany)*scale)//g_W*g_W)]

        
    def layoutplot(Canvas, Can_size, Rooms, T_Bs, Furns, Doors, Walls, Lims, showbg=True):
        Colors = {'Bedroom': 'blue', 'Kitchen': 'green', 'Livingroom': 'red', 'Toilet': 'yellow', 'Bathroom': 'purple'}
        NameFur_set = ['Bed', 'Wardrobe', 'Desk', 'Kitchen_Stove', 'Cupboard', 'Refrigerator', 'Wash_Machine',
                       'Trash_Bin', 'Dinner_Table', 'Sofa']
        Fur_set, Text = [[] for i in range(10)], [] # text is for locating the index of all text objective in canvas
        [Can_W, Can_H] = Can_size

        def scale(W, H):
            scales = [2, 2.5, 5]
            for i in range(3):
                if W/scales[i]<Can_W and H/scales[i]<Can_H: return scales[i]

        T_W, T_H = Lims[0][1]-Lims[0][0], Lims[1][1]-Lims[1][0]
        Offset = [(Lims[0][1]+Lims[0][0])//2, (Lims[1][1]+Lims[1][0])//2] #the real coordinates of the center of house
        Rotate_Flag = True if T_H>T_W else False
        Scale = scale(T_H, T_W) if Rotate_Flag else scale(T_W, T_H)
        font = 'Times 12'
        if Scale>2: font = 'Times 10' if Scale==2.5 else 'Times 7'
        T_rooms = Rooms + T_Bs
        for room in T_rooms:
            bcolor = Colors[room[-1]] if showbg else 'white'
            canxys = realxy2can(room[:4], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag) # [x0, y0, x1, y1]
            Canvas.create_rectangle(canxys[0], canxys[1], canxys[2], canxys[3], fill=bcolor, outline='')
            if room[-1] == 'Toilet' or room[-1] == 'Bathroom':
                Canvas.create_line(canxys[0], canxys[1], canxys[0], canxys[3], width=3)
                Canvas.create_line(canxys[0], canxys[1], canxys[2], canxys[1], width=3)
                Canvas.create_line(canxys[2], canxys[3], canxys[2], canxys[1], width=3)
                Canvas.create_line(canxys[2], canxys[3], canxys[0], canxys[3], width=3)
        for i, room in enumerate(Furns):
            Text.append({})
            fcolor = 'white' if showbg else 'black'
            for fur in room:
                name = fur[-1]
                canxys = realxy2can(fur[:4], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag)
                fur_Obj = Canvas.create_rectangle(canxys[0], canxys[1], canxys[2], canxys[3], outline=fcolor)
                text_Obj = Canvas.create_text((canxys[0]+canxys[2])//2, (canxys[1]+canxys[3])//2,
                                              text=Code[name], font=font, fill=fcolor)
                if name in NameFur_set:
                    j = NameFur_set.index(name)
                    Fur_set[j].append(fur_Obj)
                    Fur_set[j].append(text_Obj)
                    Text[i][name] = [text_Obj]
                elif name == 'TV':
                    Fur_set[9].append(fur_Obj)
                    Fur_set[9].append(text_Obj)
                    Text[i][name] = [text_Obj]
                elif name == 'Nightstand':
                    Fur_set[0].append(fur_Obj)
                    Fur_set[0].append(text_Obj)
                    if name not in Text[i]: Text[i][name] = [text_Obj]
                    else: Text[i][name].append(text_Obj)
                else:
                    if name not in Text[i]: Text[i][name] = [text_Obj]
                    else: Text[i][name].append(text_Obj)
                    if i == 0:
                        Fur_set[2].append(fur_Obj)
                        Fur_set[2].append(text_Obj)
                    else:
                        Fur_set[8].append(fur_Obj)
                        Fur_set[8].append(text_Obj)
        for wall in Walls:
            canxy0 = realxy2can(wall[0], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag, Is_Rect=False)
            canxy1 = realxy2can(wall[1], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag, Is_Rect=False)
            Canvas.create_line(canxy0[0], canxy0[1], canxy1[0], canxy1[1], width=3)
        for door in Doors:
            canxys = realxy2can(door[:4], Offset[0], Offset[1], Scale, Can_W, Can_H, Rotate_Flag)
            Canvas.create_line(canxys[0], canxys[1], canxys[2], canxys[3], fill='white', width=3)
        return Fur_set, Scale, Rotate_Flag, Text
    
    
    
    
    
    
    
def save_layout_figure(self, folder_path, file_name = 'Layout', show = False, save = True, close = True):
        """
        This function saves the layout data as a figure.
        
        Parameters
        ----------
        folder_path : Pathlib.Path
            folder to save the figure
        file_name : str
            file name of the layout figure (,omitting the filename extension)
        show : boolean
            whether this plt is shown
        save : boolean
            whther plt.savefig() will be done 
        close : boolean
            whether plt.close() will be done
            
        Returns
        -------
        ax : matplotlib.pyplot.gca
            Current axes on this figure.
        
        See Also
        --------
        SISG4HEI_Alpha-main/Plot/layout_plot in SISG4HEIAlpha in [2]
        """
        Toil_Bath, Furnitures, Doors, Walls, Lims \
            = self.Toil_Bath, self.Furnitures, self.Doors, self.Walls, self.Boundary
        edge = self.edge
        plt.figure()
        ax = plt.gca()
        ax.set_aspect(1)
        X_lim = [Lims[0][0] - edge, Lims[0][1] + edge]
        Y_lim = [Lims[1][0] - edge, Lims[1][1] + edge]
        plt.xlim((X_lim[0], X_lim[1]))
        plt.ylim((Y_lim[0], Y_lim[1]))
        for room in Toil_Bath:
            [x, y, L, W, name] = room
            rect = plt.Rectangle((x, y), L, W, edgecolor='k', facecolor='none')
            ax.add_patch(rect)
            ax.text(x+L//2, y+W//2, self.Code[name], fontsize = 10, va='center', ha='center')
        for room in Furnitures:
            for furniture in room:
                [x, y, L, W, name] = furniture
                rect = plt.Rectangle((x, y), L, W, edgecolor='k', facecolor='none')
                ax.add_patch(rect)
                ax.text(x+L//2, y+W//2, self.Code[name], fontsize=10, va='center', ha='center')
        for wall in Walls:
            x = [wall[0][0], wall[1][0]]
            y = [wall[0][1], wall[1][1]]
            plt.plot(x, y, 'k-')
        for door in Doors:
            x = [door[0], door[0] + door[2]]
            y = [door[1], door[1] + door[3]]
            plt.plot(x, y, 'w-')
        
        if save: plt.savefig(folder_path / (file_name + '.png'), bbox_inches = 'tight', dpi = 500)
        if show: plt.show()
        if close: plt.close()
        return ax